In [1]:
import numpy as np
import pandas
import matplotlib as mat
import seaborn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

import kagglehub
from kagglehub import KaggleDatasetAdapter

##################################################
##### STEP 1: PREPARING FEATURES AND TARGET ######
##################################################

file_path = "breast-cancer.csv"

# Load the dataset
data_frame = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "yasserh/breast-cancer-dataset",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

# print("First 5 records:\n", data_frame.head())

data_frame.dropna(inplace=True)

X = data_frame.drop(columns=["id", "diagnosis"])
y = data_frame["diagnosis"] # if this doesn't work use y = data_frame.drop(columns=X.columns)
# y = pd.Seriesy.str.replace("M", "1").str.replace("B", "0")
y.replace("M", "1", inplace=True)
y.replace("B", "0", inplace=True)

y = (pandas.DataFrame)(y)

print(f"X: {X}")
print(f"y: {y}")

C:\Users\ttjaa\Documents\College Part II\CSCI-635\csci-635_assignment-2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


X:      radius_mean  texture_mean  perimeter_mean  area_mean  smoothness_mean  \
0          17.99         10.38          122.80     1001.0          0.11840   
1          20.57         17.77          132.90     1326.0          0.08474   
2          19.69         21.25          130.00     1203.0          0.10960   
3          11.42         20.38           77.58      386.1          0.14250   
4          20.29         14.34          135.10     1297.0          0.10030   
..           ...           ...             ...        ...              ...   
564        21.56         22.39          142.00     1479.0          0.11100   
565        20.13         28.25          131.20     1261.0          0.09780   
566        16.60         28.08          108.30      858.1          0.08455   
567        20.60         29.33          140.10     1265.0          0.11780   
568         7.76         24.54           47.92      181.0          0.05263   

     compactness_mean  concavity_mean  concave points_mean  

In [2]:
##################################################
############ STEP 2: TRAIN-TEST SPLIT ############
##################################################

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)

In [3]:
##################################################
############# STEP 3: NORMALIZATION ##############
##################################################

scaler = StandardScaler().fit(X_train, y_train)

X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

print(f"X_train: {X_train}")
print(f"y_train: {y_train}")
print(f"X_val: {X_val}")
print(f"y_val: {y_val}")

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- diagnosis
Feature names seen at fit time, yet now missing:
- area_mean
- area_se
- area_worst
- compactness_mean
- compactness_se
- ...


# Why might imbalanced data be a problem?

TODO: answer the question, slick

In [ ]:
##################################################
######## STEP 4: HANDLING CLASS IMBALANCE ########
##################################################

X_resampled, y_resampled = SMOTE().fit_resample(X, y)